<a href="https://colab.research.google.com/github/Geringer13/Netology_pyda/blob/master/ML_%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_%D1%81%D0%BE%D0%B4%D0%B5%D1%80%D0%B6%D0%B0%D0%BD%D0%B8%D1%8F_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

Подготовим данные

In [9]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [0]:
data = ratings.merge(movies, how='left', on='movieId')

In [0]:
data = data.merge(tags, how='left', on=['userId', 'movieId'])

In [0]:
data = data.drop('timestamp_y', axis=1)

In [0]:
data['tag'] = data['tag'].fillna('none')

In [91]:
data.head()

,userId,movieId,rating,timestamp_x,title,genres,tag
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,none
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,none
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,none
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,none
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,none


TF IDF на тэгах и жанрах

In [35]:
# на жанрах
movie_genres = data['genres'][:20]
movie_genres

0     Adventure|Animation|Children|Comedy|Fantasy
1                                  Comedy|Romance
2                           Action|Crime|Thriller
3                                Mystery|Thriller
4                          Crime|Mystery|Thriller
5                   Action|Comedy|Horror|Thriller
6                  Adventure|Comedy|Crime|Romance
7                                Action|Drama|War
8                        Action|Drama|Romance|War
9                                      Comedy|War
10                         Action|Romance|Western
11                                         Comedy
12                                         Comedy
13                               Adventure|Comedy
14                                   Comedy|Drama
15                        Action|Adventure|Sci-Fi
16                    Comedy|Crime|Drama|Thriller
17                        Action|Adventure|Sci-Fi
18                                         Comedy
19                    Action|Crime|Drama|Thriller


In [0]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [0]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

In [0]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [38]:
neigh = NearestNeighbors(n_neighbors=7, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
                 metric_params=None, n_jobs=-1, n_neighbors=7, p=2, radius=1.0)

In [0]:
test = change_string("Adventure|Comedy|Fantasy|Crime")

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [42]:
res

(array([[0.89525066, 0.89640135, 0.93651428, 1.11958368, 1.16653668,
         1.16653668, 1.16653668]]), array([[ 6,  0, 13, 16, 18, 11, 12]]))

In [45]:
data['genres'].iloc[res[1][0]]

6                  Adventure|Comedy|Crime|Romance
0     Adventure|Animation|Children|Comedy|Fantasy
13                               Adventure|Comedy
16                    Comedy|Crime|Drama|Thriller
18                                         Comedy
11                                         Comedy
12                                         Comedy
Name: genres, dtype: object

In [0]:
# на тэгах

In [0]:
list_ = []
for i in data['tag']:
  if i not in list_:
    list_.append(i)

In [50]:
list_[:20]

['none',
 'funny',
 'Highly quotable',
 'will ferrell',
 'Boxing story',
 'MMA',
 'Tom Hardy',
 'drugs',
 'Leonardo DiCaprio',
 'Martin Scorsese',
 'way too long',
 'Al Pacino',
 'gangster',
 'mafia',
 'Mafia',
 'holocaust',
 'true story',
 'twist ending',
 'Anthony Hopkins',
 'courtroom drama']

In [51]:
movie_tags = list_[:20]
movie_tags

['none',
 'funny',
 'Highly quotable',
 'will ferrell',
 'Boxing story',
 'MMA',
 'Tom Hardy',
 'drugs',
 'Leonardo DiCaprio',
 'Martin Scorsese',
 'way too long',
 'Al Pacino',
 'gangster',
 'mafia',
 'Mafia',
 'holocaust',
 'true story',
 'twist ending',
 'Anthony Hopkins',
 'courtroom drama']

In [0]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_tags)

In [0]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [54]:
neigh = NearestNeighbors(n_neighbors=7, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
                 metric_params=None, n_jobs=-1, n_neighbors=7, p=2, radius=1.0)

In [0]:
test = change_string('funny Highly quotable will ferrell Boxing story')

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [62]:
res

(array([[1., 1., 1., 1., 1., 1., 1.]]), array([[ 8,  9, 18, 17, 16, 11, 19]]))

Строим регрессию, предсказываем оценку

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [66]:
data.head()

,userId,movieId,rating,timestamp_x,title,genres,tag
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,none
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,none
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,none
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,none
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,none


In [0]:
data = data.drop('timestamp_x', axis=1)

In [93]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102677 entries, 0 to 102676
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   102677 non-null  int64  
 1   movieId  102677 non-null  int64  
 2   rating   102677 non-null  float64
 3   title    102677 non-null  object 
 4   genres   102677 non-null  object 
 5   tag      102677 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 5.5+ MB


In [0]:
le = LabelEncoder()

In [0]:
data['title'] = le.fit_transform(data['title'])
data['genres'] = le.fit_transform(data['genres'])
data['tag'] = le.fit_transform(data['tag'])

In [96]:
data.head()

,userId,movieId,rating,title,genres,tag
0,1,1,4.0,8871,351,1167
1,1,3,4.0,3661,732,1167
2,1,6,4.0,3845,260,1167
3,1,47,5.0,7523,937,1167
4,1,50,5.0,9119,790,1167


In [0]:
X = data.drop('rating', axis=1)
y = data['rating']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [100]:
X_train.head()

,userId,movieId,title,genres,tag
89991,571,2148,4061,698,1167
50189,318,103810,7011,194,1167
30444,209,68157,4360,290,1167
65181,414,59784,4844,142,1167
55674,366,32587,7704,246,1167


In [101]:
y_train.head()

89991    2.0
50189    3.5
30444    5.0
65181    4.0
55674    3.0
Name: rating, dtype: float64

In [0]:
lin_reg = LinearRegression()

In [0]:
lin_reg.fit(X_train, y_train)
predictions = lin_reg.predict(X_test)

In [116]:
lin_reg.score(X_test, y_test)

0.00625746403022609

In [121]:
print("MAE: \t", mean_absolute_error(y_test, predictions))
print("MSE: \t", mean_squared_error(y_test, predictions))
print("R2: \t", r2_score(y_test, predictions))

MAE: 	 0.8324544353769383
MSE: 	 1.0882776637321732
R2: 	 0.00625746403022609


In [120]:
for i, j in zip(y_train, predictions):
    # c.append(int(i)- int(j))
    print('target:', i, 'pred:', j, 'diff: ', int(i)- int(j))

Streaming output truncated to the last 5000 lines.
target: 3.0 pred: 3.3977225989745943 diff:  0
target: 3.5 pred: 3.405167193151361 diff:  0
target: 3.5 pred: 3.5307113727431565 diff:  0
target: 5.0 pred: 3.5059675596745166 diff:  2
target: 4.5 pred: 3.509840158854267 diff:  1
target: 4.0 pred: 3.443551723633395 diff:  1
target: 2.5 pred: 3.557023213850016 diff:  -1
target: 4.0 pred: 3.5899299444222073 diff:  1
target: 3.5 pred: 3.5261784696254934 diff:  0
target: 3.5 pred: 3.588886290017291 diff:  0
target: 4.0 pred: 3.4806315224267257 diff:  1
target: 3.5 pred: 3.4835606686022493 diff:  0
target: 4.0 pred: 3.599665494074407 diff:  1
target: 3.0 pred: 3.3957097964067042 diff:  0
target: 5.0 pred: 3.350469858669499 diff:  2
target: 3.0 pred: 3.477965936953392 diff:  0
target: 3.5 pred: 3.453568846526166 diff:  0
target: 1.0 pred: 3.457864981411075 diff:  -2
target: 3.5 pred: 3.5149136075490315 diff:  0
target: 4.0 pred: 3.530613544358001 diff:  1
target: 4.0 pred: 3.3803976740671846 d